In [3]:
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
#import seaborn as sns
import numpy as np
%matplotlib inline

#Lectures



## Information retrieval

* collecting data mostly from user
	* activities - explicit ratings
		- star ratings
		- implementation can vary a lot, calibration (what does star means)
		- more info not especially better here
	* thumbs and like
	* reddit it is only up/down votes
	* pairwise preferences, hybrid, temporary

* when are the ratings provided?
	* consumption
	* memory
	* expectations - for v expensive items, when users don't buy it

* difficulties
	*	accuracy (noise, shifts, change in preferences)
	*	what does ratings mean?
	* more page views than feedbacks
	* use user behaviour to estimate feedback
	* user binary feedback
	* implicit data benefit from large volume and its easier to collect then explicit data
	* purchases and other user actions (implicit consumption-based)
	* follows/friends
	* how we scale and represent actions
	* respect privacy and educate

##Prediction vs recommendation

* prediction - predicts how much u like item
	* helps qualify item
* recommendation - defines items you might like, makes the suggestion
	* we are given items (list of top n) not predictions (%)
	* so it is always tie to interest or activity
	* if you don't like it you don't trust it
	* might feel too pushy - we like our privacy so soft approach might be easier

## computing rank

* how confident are you that item is good?
* what are your risk tolerances?
* specific requirements of community
* using damped means
	- $\frac{\sum_u r_{ui} +k\mu }{ n+k }$
	- without evidence everything is average
* use confidence interval
	- you recommendation is based in lower band of your interval
* use time
	- make earlier votes more important than later votes
	- this will decrease influence of new votes for highly voted items
* damping results so we can influence results for community interest sake 





# Assigment